## Notebook for using DenseNet to tell if the MCA is visible for a slice
Pipeline:
- Import data
- Find volume with the highest cumulative intensity for each patient
- Discard all other volumes
- Perform train, val, test split
- Normalize data <- todo?
- Initialize model
- Train model
- Test model
- Visualize results
- Use KMeans to extract AIFs

In [1]:
# Imports
import numpy as np
import pandas as pd
import pydicom as pdc
from preprocess_utils import createImageIndexCSV, get_train_test_split_on_patients, get_max_intensity_for_dataset
from dense_net import DenseNet3
from model_utils import train_and_eval

SEED = 41
BATCH_SIZE = 8
image_path = "D:/iCAT_IMAGES"
aif_path = "D:/AIFs/AIFs/durable/BorrSci_MR_Data/Output"

In [2]:
# Import images (paths)
image_data = createImageIndexCSV(image_path)
# Import annotations
mca_labels = pd.read_excel('MCA_labels.xlsx')
mca_labels = mca_labels[:47] # Not all patients are annotated
mca_labels = mca_labels.drop(mca_labels[mca_labels['Patient'] == 11].index) # Patient 11 is missing a file
image_data = image_data[image_data['Patient'].isin(mca_labels['Patient'])]

In [3]:
# Change the label dataframe to be compatible with efficient datahandling
mca_labels = pd.melt(mca_labels, id_vars = ['Patient'], var_name='Slice', value_name='Label').sort_values(['Patient', 'Slice'])

In [9]:
vol_intensities = get_max_intensity_for_dataset(image_data)

Here


In [5]:
# Only keep the relevant volumes
extra_vols = 3
copy_data = image_data.copy()
for patient, volume_index in zip(vol_intensities[:, 0], vol_intensities[:, 1]):
    copy_data = copy_data.drop(copy_data[copy_data['Patient'] == int(patient)].index & copy_data[copy_data['Volume'] != int(volume_index)].index)
    if extra_vols + volume_index < 48:
        for i in range(int(volume_index)+1, extra_vols +int(volume_index) +1):
            extra_data = image_data[image_data['Slice'].isin(mca_labels[(mca_labels['Patient'] == patient) & (mca_labels['Label'] == 1)]['Slice'])]
            copy_data = pd.concat([copy_data, extra_data[extra_data['Volume'] == i]])


C:\Users\Sven Alrik\AppData\Local\Temp\ipykernel_16944\1690949236.py:5: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead.
  copy_data = copy_data.drop(copy_data[copy_data['Patient'] == int(patient)].index & copy_data[copy_data['Volume'] != int(volume_index)].index)
C:\Users\Sven Alrik\AppData\Local\Temp\ipykernel_16944\1690949236.py:5: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead.
  copy_data = copy_data.drop(copy_data[copy_data['Patient'] == int(patient)].index & copy_data[copy_data['Volume'] != int(volume_index)].index)
C:\Users\Sven Alrik\AppData\Local\Temp\ipykernel_16944\1690949236.py:5: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matchi

In [7]:
train_images, test_images = get_train_test_split_on_patients(copy_data)

In [8]:
train_labels = np.isin(mca_labels['Patient'], np.unique(train_images['Patient']))
train_labels = mca_labels[train_labels]
test_labels = np.isin(mca_labels['Patient'], np.unique(test_images['Patient']))
test_labels = mca_labels[test_labels]

In [10]:
label_train_true_size, label_train_size = len(train_labels[train_labels['Label'] == 1]), len(train_labels)
label_test_true_size, label_test_size = len(test_labels[test_labels['Label'] == 1]), len(test_labels)
print(f"Size training data: {len(train_images)}. Size test_data: {len(test_images)}")
print(f"Number of true labels in training: {label_train_true_size} of {label_train_size} ({round(label_train_true_size/label_train_size, 3)*100}%). True labels in test: {label_test_true_size} of {label_test_size} ({round(label_test_true_size/label_test_size, 3)*100}%)")

Size training data: 3139. Size test_data: 862
Number of true labels in training: 66 of 864 (7.6%). True labels in test: 20 of 240 (8.3%)


In [11]:
from preprocess_utils import SliceIntensityDataset
train_dataset, test_dataset = SliceIntensityDataset(train_images, train_labels), SliceIntensityDataset(test_images, test_labels)

In [12]:
from torch.utils.data import DataLoader
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [13]:
model = DenseNet3(64, num_classes=1, dropRate=0.2)
print(f"Total number of trainable parameters {sum(p.numel() for p in model.parameters() if p.requires_grad)}")

Total number of trainable parameters 365737


In [14]:
import torch
torch.cuda.empty_cache()

In [15]:
trained_model, losses = train_and_eval(model, test_loader, train_loader, 10)

Epoch 1 of 2
Losses: 8.031060218811035
Epoch 2 of 2


KeyboardInterrupt: 

In [14]:
def get_model_performance_metrics(model, images, labels):
    # Copy dataframe due to the use of iterrows
    images_copy = images.copy()
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    for _, selected in images_copy.iterrows():
        model.eval()
        image = pdc.read_file(selected['ImagePath'])
        image = torch.tensor(np.expand_dims(image.pixel_array.astype("int32"), axis=(0,1)), dtype=torch.float32)
        image = image.to('cuda')
        actual = labels[labels['Patient/slice'] == int(selected['Patient'])][int(selected['Slice'])].item()
        predicted = int(round(torch.sigmoid(model(image)).item()))
        if predicted == 1 and actual == 1:
            TP += 1
        elif predicted == 1 and actual == 0:
            FP += 1
        elif predicted == 0 and actual == 0:
            TN += 1
        elif predicted == 0 and actual == 1:
            FN += 1
        else:
            pass
    print(f"TP: {TP}, FP: {FP}, TN: {TN}, FN: {FN}")
    print(f"Accuracy: {round((TP+TN)/(TP+FP+TN+FN),2)}, Precision: {round(TP/(FP+TP), 2)}, Recall: {round(TP/(TP+FN), 2)}")

print("Performance on training data:")
get_model_performance_metrics(trained_model, train_images, train_labels)
print("Performance on test data:")
get_model_performance_metrics(trained_model, test_images, test_labels)


Performance on training data:
TP: 280, FP: 6, TN: 4395, FN: 23
Accuracy: 0.99, Precision: 0.98, Recall: 0.92
Performance on test data:
TP: 10, FP: 5, TN: 215, FN: 10
Accuracy: 0.94, Precision: 0.67, Recall: 0.5


In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
plt.plot(losses)